### Pulling and Storing Fundamentals Data
To set up WRDS in python run the following:

    import wrds
    db = wrds.Connection(wrds_username='islipdav')
    db.create_pgpass_file()

it will ask you for your username and password and will set up your password file on your local machine

In [1]:
from services.database import Database
from services.dataloading import *
Database.initialize()
ticker_info = Database.query("SELECT * from ticker_index_membership "
                             "where equity_index = 'S&P500' and "
                             "startDate <> '1900-01-01 00:00:00.000000'")

ticker_info['ticker_currency'] = ticker_info.ticker + "-" + ticker_info.priceCurrency

In [3]:
len(ticker_info.ticker.unique()) == len(ticker_info)

False

In [150]:
raw_data = get_raw_wrds_ratios('islipdav', ticker_info)

Loading library list...
Done


In [187]:
de_duped_df_out = process_wrds_ratio_data(raw_data, ticker_info)

In [168]:
#add the metrics to the id table
add_to_metric_id(Database, de_duped_df_out.metric.unique(), desc = 'see WRDS documentation for more details')

In [195]:
#add the data to the fundamentals data
wrds_funda_merged = merge_fundamentals_df(Database, de_duped_df_out)
add_dfs_to_db('fundamentals', Database, [wrds_funda_merged], indices_of_primary_key=[0])

### ML Factor Fundamentals Dataset

In [20]:
def process_ml_factor_fundamentals():
    ml_factor = pd.read_csv("local data/mlfactor/data_ml.csv")
    ml_factor['ticker'] = 'MLBook' + ml_factor.stock_id.astype('string')
    ml_factor = ml_factor.drop(['Unnamed: 0', 'stock_id'], axis = 1)
    ml_factor_long = ml_factor.melt( id_vars = ['ticker', 'date'])
    ml_factor_long['frequency'] = 'M'
    ml_factor_long['source'] = 'mlfactor'
    ml_factor_long = ml_factor_long.rename(columns={"variable": "metric"})

    group_cols = ['ticker', 'date', 'metric', 'frequency', 'source']
    de_dup_df = replace_duplicates_with_mean(group_cols, ml_factor_long)

    de_dup_df['priceCurrency'] = 'USD'
    de_dup_df['ticker_currency'] = de_dup_df.ticker + "-" + de_dup_df.priceCurrency
    de_dup_df.date = pd.to_datetime(de_dup_df.date, format="%Y-%m-%d")

    return de_dup_df

In [23]:
ml_factor_fundamentals = process_ml_factor_fundamentals()
ml_factor_fundamentals

,ticker,date,metric,frequency,source,value,priceCurrency,ticker_currency
0,MLBook13,2006-12-31,Advt_12M_Usd,M,mlfactor,0.250,USD,MLBook13-USD
1,MLBook13,2007-01-31,Advt_12M_Usd,M,mlfactor,0.250,USD,MLBook13-USD
2,MLBook13,2007-02-28,Advt_12M_Usd,M,mlfactor,0.260,USD,MLBook13-USD
3,MLBook17,2015-03-31,Advt_12M_Usd,M,mlfactor,0.730,USD,MLBook17-USD
4,MLBook17,2015-04-30,Advt_12M_Usd,M,mlfactor,0.720,USD,MLBook17-USD
...,...,...,...,...,...,...,...,...
27487855,MLBook1205,2004-05-31,R12M_Usd,M,mlfactor,0.112,USD,MLBook1205-USD
27487856,MLBook1205,2004-07-31,R12M_Usd,M,mlfactor,0.122,USD,MLBook1205-USD
27487857,MLBook1205,2004-08-31,R12M_Usd,M,mlfactor,0.091,USD,MLBook1205-USD
27487858,MLBook1205,2004-09-30,R12M_Usd,M,mlfactor,0.040,USD,MLBook1205-USD


In [24]:
add_to_metric_id(Database, ml_factor_fundamentals.metric.unique(), desc = 'see ML for Factor textbook')

In [25]:
ml_factor_funda_merged = merge_fundamentals_df(Database, ml_factor_fundamentals)

In [26]:
ml_factor_funda_merged

,ticker_x,date,metric,frequency,source,value,priceCurrency,ticker_currency,source_id,description,frequency_id,metric_id,desc,ticker_id,ticker_y,currency,id
0,MLBook13,2006-12-31,Advt_12M_Usd,M,mlfactor,0.25,USD,MLBook13-USD,2,Machine learning for factor investing,1,0,see ML for Factor textbook,106563,MLBook13,USD,21010656320061231
1,MLBook13,2007-01-31,Advt_12M_Usd,M,mlfactor,0.25,USD,MLBook13-USD,2,Machine learning for factor investing,1,0,see ML for Factor textbook,106563,MLBook13,USD,21010656320070131
2,MLBook13,2007-02-28,Advt_12M_Usd,M,mlfactor,0.26,USD,MLBook13-USD,2,Machine learning for factor investing,1,0,see ML for Factor textbook,106563,MLBook13,USD,21010656320070228
3,MLBook13,2005-06-30,Advt_12M_Usd,M,mlfactor,0.20,USD,MLBook13-USD,2,Machine learning for factor investing,1,0,see ML for Factor textbook,106563,MLBook13,USD,21010656320050630
4,MLBook13,2005-07-31,Advt_12M_Usd,M,mlfactor,0.20,USD,MLBook13-USD,2,Machine learning for factor investing,1,0,see ML for Factor textbook,106563,MLBook13,USD,21010656320050731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27487855,MLBook950,2018-09-30,R12M_Usd,M,mlfactor,0.00,USD,MLBook950-USD,2,Machine learning for factor investing,1,96,see ML for Factor textbook,107769,MLBook950,USD,219610776920180930
27487856,MLBook950,2018-10-31,R12M_Usd,M,mlfactor,0.00,USD,MLBook950-USD,2,Machine learning for factor investing,1,96,see ML for Factor textbook,107769,MLBook950,USD,219610776920181031
27487857,MLBook950,2018-11-30,R12M_Usd,M,mlfactor,0.00,USD,MLBook950-USD,2,Machine learning for factor investing,1,96,see ML for Factor textbook,107769,MLBook950,USD,219610776920181130
27487858,MLBook950,2018-12-31,R12M_Usd,M,mlfactor,0.00,USD,MLBook950-USD,2,Machine learning for factor investing,1,96,see ML for Factor textbook,107769,MLBook950,USD,219610776920181231


In [27]:
add_dfs_to_db('fundamentals', Database, [ml_factor_funda_merged], indices_of_primary_key=[0])